## Importing necessary libraries

In [ ]:
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import plot_model
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
from sklearn.model_selection import train_test_split

## Spliting the data folder into : Train and Test 

In [ ]:
# Set the path to your data directory
data_dir = "C:/Users/DELL/Desktop/datatest/D-E"

# Get the list of all folders names
image_files = os.listdir(data_dir)
print("All folders names:", image_files)

In [ ]:
# Get the list of all pass image file names
pass_dir = os.path.join(data_dir, "pass")
pass_images = [os.path.join(pass_dir, file) for file in os.listdir(pass_dir)]

# Print the lists of pass and fail image file names
print("Pass images:", pass_images)

In [ ]:
# Get the list of all fail image file names
fail_dir = os.path.join(data_dir, "fail")
fail_images = [os.path.join(fail_dir, file) for file in os.listdir(fail_dir)]

# Print the lists of pass and fail image file names
print("Fail images:", fail_images)

In [ ]:
# Split the data into train and test sets (70% train, 30% test)
pass_train, pass_test = train_test_split(pass_images, test_size=0.3, random_state=42)
fail_train, fail_test = train_test_split(fail_images, test_size=0.3, random_state=42)

# Create the train and test directories
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
os.makedirs(os.path.join(train_dir, "pass"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "fail"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "pass"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "fail"), exist_ok=True)

# Copy the pass images to the train and test directories
for file in pass_train:
    dest_file = os.path.join(train_dir, "pass", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)
for file in pass_test:
    dest_file = os.path.join(test_dir, "pass", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)

# Copy the fail images to the train and test directories
for file in fail_train:
    dest_file = os.path.join(train_dir, "fail", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)
for file in fail_test:
    dest_file = os.path.join(test_dir, "fail", os.path.basename(file))
    if not os.path.exists(dest_file):
        shutil.copy(file, dest_file)

## Define the model architecture

In [ ]:
# Define the input image size and number of classes
img_width = 224
img_height = 224
batch_size = 32
num_classes = 2

In [ ]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build a new model on top of the pre-trained VGG16 model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Print the model summary
model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the data generators for image augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

In [ ]:
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Train the model
history = model.fit(
    train_data, 
    epochs=1000, 
    steps_per_epoch=train_data.n // train_data.batch_size,
    validation_data=test_data,
    validation_steps=test_data.n // test_data.batch_size,
    callbacks=[early_stop]
)

## Plots of Training and Validation Accuracy/Loss for the Model

In [ ]:
# Set the Seaborn style
sns.set_style("darkgrid")
# Plot the training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Set the Seaborn style
sns.set_style("darkgrid")
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Save the accuracy plot to a file
plt.savefig('model_vgg16_accuracy.png')
# Save the accuracy plot to a file
plt.savefig('model_vgg16_loss.png')